In [1]:
import pandas as pd
import refinitiv.data as rd
from datetime import datetime, timedelta
from pydash.arrays import chunk
from tqdm import tqdm
import numpy as np

rd.open_session(app_key= '')

<refinitiv.data.session.Definition object at 0x264a95cf1d0 {name='workspace'}>

In [2]:
def merge_and_split(year):
    
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    
    cik_df = pd.read_parquet(f"Data/{year}/cik_{year}.parquet")
    ric_df = pd.read_parquet(f"Data/{year}/ric_{year}.parquet")
    
    # Convert the 'cik' column in both DataFrames to strings
    cik_df['cik'] = cik_df['cik'].astype(str)
    ric_df['cik'] = ric_df['cik'].astype(str).str.zfill(10)
    
    # Perform a right merge so that all rows from ric_df are kept
    merged_df = pd.merge(cik_df, ric_df, how='right', on='cik')
    
    # Turn RIC column into string
    merged_df['RIC'] = merged_df['RIC'].astype(str)
    
    with open(f'Data/{year}/{year}_Stockprices/cik_and_ric_analysis.txt', "a") as f:
        f.write(f"Merged CIK and RIC {year} NA analysis:")
        f.write(f"{merged_df.isna().sum()}")
        f.write("\n")
   
    # Split the DataFrame into 3 parts
    df_parts = np.array_split(merged_df, 3)
    
    # Assign each part to a variable (optional)
    df_part1, df_part2, df_part3 = df_parts
    
    with open(f'Data/{year}/{year}_Stockprices/cik_and_ric_analysis.txt', "a") as f:
        f.write(f"Dataframe was split into 3 parts:\n")
        f.write(f"Part 1: {len(df_part1)} rows\n")
        f.write(f"Part 2: {len(df_part2)} rows\n")
        f.write(f"Part 3: {len(df_part3)} rows\n")
        f.write(f"All three parts added together are as big as the original: {len(df_part1) + len(df_part2) + len(df_part3) == len(merged_df)}")
        
    return df_parts

In [3]:
def get_stockprices(ric_date_df, year, part_number):

    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    results = []  # To store individual DataFrames
    
    # Iterate through the rows of the input DataFrame with tqdm
    # for _, row in tqdm(ric_date_df.iterrows(), desc="Processing batches"): ## old code
        
    for _, row in tqdm(
        ric_date_df.iterrows(), 
        total=ric_date_df.shape[0],         # Explicit total for accurate progress tracking
        desc="Processing batches",          # Custom description for the progress bar
        unit="batch",                       # Custom unit name to display per iteration
        ncols=100,                          # Width of the progress bar (optional)
        mininterval=0.1): 


        extracted_date = row['full_date']
        extracted_ric = row['RIC']
        
        # First we construct the time interval
        date_object = extracted_date

        # Subtract 250 days
        date_object_minus_250 = date_object - timedelta(days=250)

        # Convert the new dates to an ISO8601 string
        event_date = date_object.strftime("%Y-%m-%d")
        date_minus_250 = date_object_minus_250.strftime("%Y-%m-%d")
        
        
        # Call get_history for the current batch        
        try:
            Stockprices_dataframe = rd.get_history(universe= extracted_ric, fields=[ "TR.PriceClose"], interval="1D",
               start = date_minus_250, end = event_date)         
            
            # Turn index 'date' into a column 
            Stockprices_dataframe_with_date = Stockprices_dataframe.reset_index()   
            
            # Add the RIC column to the resulting DataFrame
            Stockprices_dataframe_with_date['RIC'] = extracted_ric
            
            # Append DataFrame to results
            results.append(Stockprices_dataframe_with_date)  
            
        except Exception as e:
            print(f"Error processing batch {row}")


    # Concatenate all results into a single DataFrame
    result = pd.concat(results, ignore_index=True)
    
        # Save the cleaned DataFrame
    result.to_parquet(f'Data/{year}/{year}_Stockprices/{year}_Stockprices_part_{part_number}.parquet', engine='fastparquet')
    
    print(f"Stockprices for {year} part {part_number } has been analyzed and saved into a parquet file.")


In [4]:
def get_stockprice_data(year):
    df = merge_and_split(year)
    
    part_counter = 0
    
    for part in df:
        part_counter += 1
        get_stockprices(ric_date_df= part, year=year, part_number=part_counter)
        
    print(f"All parts of {year} have been succesfully analyzed and saved into parquet files.")

In [37]:
get_stockprice_data("2016")

Processing batches:  98%|█████████████████████████████████████▎| 785/800 [07:56<00:08,  1.71batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|██████████████████████████████████████| 800/800 [08:05<00:00,  1.65batch/s]


Stockprices for 2016 part 1 has been analyzed and saved into a parquet file.


Processing batches:  50%|███████████████████                   | 401/799 [04:09<04:04,  1.63batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|██████████████████████████████████████| 799/799 [08:19<00:00,  1.60batch/s]


Stockprices for 2016 part 2 has been analyzed and saved into a parquet file.


Processing batches:  21%|███████▉                              | 167/799 [01:44<05:47,  1.82batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|██████████████████████████████████████| 799/799 [08:03<00:00,  1.65batch/s]

Stockprices for 2016 part 3 has been analyzed and saved into a parquet file.
All parts of 2016 have been succesfully analyzed and saved into parquet files.


In [38]:
get_stockprice_data("2017")

Processing batches: 100%|██████████████████████████████████████| 782/782 [08:21<00:00,  1.56batch/s]


Stockprices for 2017 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 782/782 [11:10<00:00,  1.17batch/s]


Stockprices for 2017 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 781/781 [11:06<00:00,  1.17batch/s]

Stockprices for 2017 part 3 has been analyzed and saved into a parquet file.
All parts of 2017 have been succesfully analyzed and saved into parquet files.


In [39]:
get_stockprice_data("2018")

Processing batches:  99%|█████████████████████████████████████▋| 773/779 [10:58<00:04,  1.45batch/s]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|██████████████████████████████████████| 779/779 [11:02<00:00,  1.18batch/s]


Stockprices for 2018 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 778/778 [11:31<00:00,  1.13batch/s]


Stockprices for 2018 part 2 has been analyzed and saved into a parquet file.


Processing batches:  44%|████████████████▋                     | 341/778 [04:55<07:57,  1.09s/batch]C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
C:\Users\kcthi\Economic Data Science\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast
Processing batches: 100%|██████████████████████████████████████| 778/778 [11:15<00:00,  1.15batch/s]

Stockprices for 2018 part 3 has been analyzed and saved into a parquet file.
All parts of 2018 have been succesfully analyzed and saved into parquet files.


In [40]:
get_stockprice_data("2019")

Processing batches: 100%|██████████████████████████████████████| 789/789 [11:31<00:00,  1.14batch/s]


Stockprices for 2019 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 788/788 [11:47<00:00,  1.11batch/s]


Stockprices for 2019 part 2 has been analyzed and saved into a parquet file.


Processing batches:  40%|███████████████▎                      | 318/788 [05:17<00:37, 12.42batch/s]

...

Error processing batch cik                   0001314727
full_date    2019-11-25 00:00:00
RIC                      SONO.OQ
RIC_clean                SONO.OQ
Name: 2252, dtype: object
Error processing batch cik                   0001476765
full_date    2019-11-25 00:00:00
RIC                      GBDC.OQ
RIC_clean                GBDC.OQ
Name: 2253, dtype: object
Error processing batch cik                   0000108312
full_date    2019-11-25 00:00:00
RIC                       WWD.OQ
RIC_clean                 WWD.OQ
Name: 2254, dtype: object
Error processing batch cik                   0001222333
full_date    2019-11-25 00:00:00
RIC                        GLD.P
RIC_clean                  GLD.P
Name: 2255, dtype: object
Error processing batch cik                   0000068709
full_date    2019-11-25 00:00:00
RIC                  MTSC.OQ^D21
RIC_clean                MTSC.OQ
Name: 2256, dtype: object
Error processing batch cik                   0000879407
full_date    2019-11-25 00:00:00
RIC   

Processing batches:  95%|██████████████████████████████████▉  | 745/788 [05:19<00:00, 208.38batch/s]

Error processing batch cik                   0001527613
full_date    2019-12-26 00:00:00
RIC                       IMG.OQ
RIC_clean                 IMG.OQ
Name: 2300, dtype: object
Error processing batch cik                   0000040570
full_date    2019-12-23 00:00:00
RIC                        JOB.A
RIC_clean                  JOB.A
Name: 2301, dtype: object
Error processing batch cik                   0000836690
full_date    2019-12-23 00:00:00
RIC                      ISSC.OQ
RIC_clean                ISSC.OQ
Name: 2302, dtype: object
Error processing batch cik                   0000810136
full_date    2019-12-20 00:00:00
RIC                      PLAB.OQ
RIC_clean                PLAB.OQ
Name: 2303, dtype: object
Error processing batch cik                   0000010048
full_date    2019-12-20 00:00:00
RIC                        BRN.A
RIC_clean                  BRN.A
Name: 2304, dtype: object
Error processing batch cik                   0001681622
full_date    2019-12-20 00:00:00
RIC   

Processing batches: 100%|██████████████████████████████████████| 788/788 [05:19<00:00,  2.47batch/s]

Error processing batch cik                   0001589526
full_date    2019-12-12 00:00:00
RIC                      BLBD.OQ
RIC_clean                BLBD.OQ
Name: 2343, dtype: object
Error processing batch cik                   0000863894
full_date    2019-12-12 00:00:00
RIC                      VERU.OQ
RIC_clean                VERU.OQ
Name: 2344, dtype: object
Error processing batch cik                   0000913760
full_date    2019-12-11 00:00:00
RIC                      SNEX.OQ
RIC_clean                SNEX.OQ
Name: 2345, dtype: object
Error processing batch cik                   0000785557
full_date    2019-12-11 00:00:00
RIC                      DLHC.OQ
RIC_clean                DLHC.OQ
Name: 2346, dtype: object
Error processing batch cik                   0001616741
full_date    2019-12-11 00:00:00
RIC                  PATI.OQ^L23
RIC_clean                PATI.OQ
Name: 2347, dtype: object
Error processing batch cik                   0000064472
full_date    2019-12-11 00:00:00
RIC   

In [5]:
get_stockprice_data("2020")

Processing batches: 100%|██████████████████████████████████████| 803/803 [12:00<00:00,  1.12batch/s]


Stockprices for 2020 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 803/803 [10:08<00:00,  1.32batch/s]


Stockprices for 2020 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 803/803 [09:35<00:00,  1.39batch/s]

Stockprices for 2020 part 3 has been analyzed and saved into a parquet file.
All parts of 2020 have been succesfully analyzed and saved into parquet files.


In [6]:
get_stockprice_data("2021")

Processing batches: 100%|██████████████████████████████████████| 893/893 [10:53<00:00,  1.37batch/s]


Stockprices for 2021 part 1 has been analyzed and saved into a parquet file.


Processing batches:  98%|█████████████████████████████████████ | 870/892 [14:16<02:11,  5.99s/batch]

Error processing batch cik                   0001747748
full_date    2021-03-09 00:00:00
RIC                    XM.OQ^F23
RIC_clean                  XM.OQ
Name: 1762, dtype: object


Processing batches: 100%|██████████████████████████████████████| 892/892 [14:36<00:00,  1.02batch/s]


Stockprices for 2021 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 892/892 [10:59<00:00,  1.35batch/s]

Stockprices for 2021 part 3 has been analyzed and saved into a parquet file.
All parts of 2021 have been succesfully analyzed and saved into parquet files.


In [7]:
get_stockprice_data("2022")

Processing batches: 100%|████████████████████████████████████| 1035/1035 [13:22<00:00,  1.29batch/s]


Stockprices for 2022 part 1 has been analyzed and saved into a parquet file.


Processing batches:  39%|██████████████▌                      | 407/1035 [04:58<05:18,  1.97batch/s]

Error processing batch cik                   0001893219
full_date    2022-03-24 00:00:00
RIC                       ABP.OQ
RIC_clean                 ABP.OQ
Name: 1441, dtype: object


Processing batches: 100%|████████████████████████████████████| 1035/1035 [12:56<00:00,  1.33batch/s]


Stockprices for 2022 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|████████████████████████████████████| 1035/1035 [13:34<00:00,  1.27batch/s]

Stockprices for 2022 part 3 has been analyzed and saved into a parquet file.
All parts of 2022 have been succesfully analyzed and saved into parquet files.


In [8]:
get_stockprice_data("2023")

Processing batches:  78%|█████████████████████████████▌        | 766/984 [09:57<01:48,  2.01batch/s]

Error processing batch cik                   0001893219
full_date    2023-03-31 00:00:00
RIC                       ABP.OQ
RIC_clean                 ABP.OQ
Name: 765, dtype: object


Processing batches: 100%|██████████████████████████████████████| 984/984 [12:43<00:00,  1.29batch/s]


Stockprices for 2023 part 1 has been analyzed and saved into a parquet file.


Processing batches:  88%|█████████████████████████████████▎    | 861/984 [09:54<00:04, 28.32batch/s]

...

Processing batches:  98%|████████████████████████████████████▏| 962/984 [00:04<00:00, 216.44batch/s]

Error processing batch cik                   0001435508
full_date    2023-12-20 00:00:00
RIC                      FSFG.OQ
RIC_clean                FSFG.OQ
Name: 2907, dtype: object
Error processing batch cik                   0000072331
full_date    2023-12-20 00:00:00
RIC                      NDSN.OQ
RIC_clean                NDSN.OQ
Name: 2908, dtype: object
Error processing batch cik                   0001000230
full_date    2023-12-20 00:00:00
RIC                       OCC.OQ
RIC_clean                 OCC.OQ
Name: 2909, dtype: object
Error processing batch cik                   0000886128
full_date    2023-12-19 00:00:00
RIC                      FCEL.OQ
RIC_clean                FCEL.OQ
Name: 2910, dtype: object
Error processing batch cik                   0001882078
full_date    2023-12-18 00:00:00
RIC                      FUFU.OQ
RIC_clean                FUFU.OQ
Name: 2911, dtype: object
Error processing batch cik                   0000040570
full_date    2023-12-18 00:00:00
RIC   

Processing batches: 100%|█████████████████████████████████████| 984/984 [00:04<00:00, 199.60batch/s]


Error processing batch cik                   0001069533
full_date    2023-12-01 00:00:00
RIC                      RGCO.OQ
RIC_clean                RGCO.OQ
Name: 2951, dtype: object


ValueError: No objects to concatenate

In [5]:
get_stockprice_data("2019")

Processing batches: 100%|██████████████████████████████████████| 789/789 [09:40<00:00,  1.36batch/s]


Stockprices for 2019 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 788/788 [11:09<00:00,  1.18batch/s]


Stockprices for 2019 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 788/788 [11:53<00:00,  1.10batch/s]

Stockprices for 2019 part 3 has been analyzed and saved into a parquet file.
All parts of 2019 have been succesfully analyzed and saved into parquet files.


In [6]:
get_stockprice_data("2023")

Processing batches:  78%|█████████████████████████████▌        | 766/984 [11:40<01:46,  2.04batch/s]

Error processing batch cik                   0001893219
full_date    2023-03-31 00:00:00
RIC                       ABP.OQ
RIC_clean                 ABP.OQ
Name: 765, dtype: object


Processing batches: 100%|██████████████████████████████████████| 984/984 [13:55<00:00,  1.18batch/s]


Stockprices for 2023 part 1 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 984/984 [10:04<00:00,  1.63batch/s]


Stockprices for 2023 part 2 has been analyzed and saved into a parquet file.


Processing batches: 100%|██████████████████████████████████████| 984/984 [09:52<00:00,  1.66batch/s]

Stockprices for 2023 part 3 has been analyzed and saved into a parquet file.
All parts of 2023 have been succesfully analyzed and saved into parquet files.


In [1]:
import pandas as pd

In [4]:
# For one df per year 

def check_stock_output(year):
    
    df = pd.read_parquet(f"Data/{year}/Regression_Target_{year}/{year}_Regression_Stockprices.parquet") 

    unique_values = df['RIC'].unique()

    size_all_companies = len(unique_values) 
    
    with open(f'Data/Stockprice_Data_for_Regression.txt', "a") as f:
        f.write(f"In {year} we have stock prices for {size_all_companies} companies\n")
        f.write(f"Number of missing stockprices for {year}: {df['Price Close'].isna().sum()}\n")
        f.write("===================================================================================\n")

In [7]:
# For parts

def check_stock_output(year):
    
    df_part_1 = pd.read_parquet(f"Data/{year}/{year}_Stockprices/{year}_Stockprices_part_1.parquet") 
    df_part_2 = pd.read_parquet(f"Data/{year}/{year}_Stockprices/{year}_Stockprices_part_2.parquet") 
    df_part_3 = pd.read_parquet(f"Data/{year}/{year}_Stockprices/{year}_Stockprices_part_3.parquet") 
    
    unique_values_part_1 = df_part_1['RIC'].unique()
    unique_values_part_2 = df_part_2['RIC'].unique()
    unique_values_part_3 = df_part_3['RIC'].unique()
    
    size_all_companies = len(unique_values_part_1) + len(unique_values_part_2) + len(unique_values_part_3)
    
    # print(f"In {year} we have stock prices for {size_all_companies} companies")
    # print(f"Number of missing stockprices for part 1: {df_part_1['Price Close'].isna().sum()}")
    # print(f"Number of missing stockprices for part 2: {df_part_2['Price Close'].isna().sum()}")
    # print(f"Number of missing stockprices for part 3: {df_part_3['Price Close'].isna().sum()}")
    
    
    with open(f'Data/Stockprice_Data.txt', "a") as f:
        f.write(f"In {year} we have stock prices for {size_all_companies} companies\n")
        f.write(f"Number of missing stockprices for part 1: {df_part_1['Price Close'].isna().sum()}\n")
        f.write(f"Number of missing stockprices for part 2: {df_part_2['Price Close'].isna().sum()}\n")
        f.write(f"Number of missing stockprices for part 3: {df_part_3['Price Close'].isna().sum()}\n")
        f.write("===================================================================================\n")

In [5]:
years = ["2016", "2017", "2018","2019", "2020", "2021", "2022", "2023"]
for year in years:
    check_stock_output(year)
    